**0. Importing dependencies**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from bs4 import BeautifulSoup #requires pip install
import requests #requires pip install
import re

import time

**1. Scraper setup for dexters.co.uk**

In [5]:
# document time 
time_started = str(datetime.datetime.now()).replace(" ","_").replace(":","-")[0:19]

In [6]:
## Define list of subway stations
Underground_lines = ['Bakerloo', 'Central', 'Circle', 'District', 'DLR', 'Hammersmith & City',
                     'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']

1.1 Functions to scrape information from main add page, only related to price, address, and features of the add

In [7]:
## Function to extract characteristics on each ad from the main webpage
def feature_extract(html_text):
    
    soup = BeautifulSoup(html_text, 'lxml')

    ## Parse for the different divisions within the add
    ads = soup.find_all('li', class_ = 'result item for-sale infinite-item') #searches for 'div' and is filtered by the CSS-snippet

    ## Set-up for the loop 
    results = {} #create nested dictionary to store the results
    id_ad = 0 #insert ad_ID to distinguish between each ad 

    ## Loop across all ads
    for k in range(len(ads)):
        ad = ads[k]
        id_ad += 1
        results[id_ad] = {}

        ## Extracting features from the ad
        name = ad.find('h3').a.contents[0]
        try:
            price = ad.find('span', class_ = 'price-qualifier').text #catches the price WITHIN one ad
        except:
            continue
        address = ad.find('span', class_ = 'address-area-post').text

        # Number of bedrooms extracted from string
        try:
            bedrooms = ad.find('li', class_ = 'Bedrooms').text
        except:
            continue
        bedrooms_nbr = int(bedrooms.split()[0])

        # Number of bedrooms extracted from string
        bathrooms_str = str(ad.find('li',class_ = 'Bathrooms'))
        bathrooms_nbr = re.findall(r'\d+', bathrooms_str)
        bathrooms_nbr2 = int(bathrooms_nbr[0] if len(bathrooms_nbr)!= 0  else 0)

        # Number of bedrooms extracted from string
        reception_str = str(ad.find('li',class_ = 'Receptions'))
        reception_nbr = re.findall(r'\d+', reception_str)
        reception_nbr2 = int(reception_nbr[0] if len(reception_nbr)!= 0  else 1)

        link = ad.find('h3').a.get("href")
        ad_identification = ads[k]['data-property-id']

        # Create dictionary of results per ad id
        results[id_ad]["ad_identification"] = ad_identification
        results[id_ad]["name"] = name
        results[id_ad]["price"] = price
        results[id_ad]["address"] = address
        results[id_ad]["bedrooms"] = bedrooms_nbr
        results[id_ad]["bathrooms"] = bathrooms_nbr2
        results[id_ad]["reception"] = reception_nbr2
        results[id_ad]["link"] = ("https://www.dexters.co.uk" + link)
        
        # Create dataframe from dictionary of results
        df_houses = pd.DataFrame.from_dict(results, orient='index')

    return df_houses

In [8]:
url = "https://www.dexters.co.uk/property-sales/properties-for-sale-in-london"
html_text = requests.get(url).text

feature_extract(html_text).head(5)

,ad_identification,name,price,address,bedrooms,bathrooms,reception,link
1,151907,Lancaster Gate,"£25,000,000","Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...
2,124656,Wilton Crescent,"£19,500,000","Belgravia, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...
3,138546,Whitehall Place,"£9,250,000","Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...
4,143179,Old Queen Street,"£9,250,000","Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...
5,124663,Wilton Place,"£9,000,000","Belgravia, SW1X",5,5,3,https://www.dexters.co.uk/property-for-sale/ho...


In [9]:
## Function to create list of pages base on url and number of iterations desired
def page_list(string, iterations):
    pages_list = []
    for i in range(iterations):
        pages_list.append(string + str(i+1))
        
    return pages_list

In [10]:
## Function to get the maximum number of listing on Dexter's website
def page_max(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    amount = soup.find('span', class_ = 'marker-count has-results').text
    amount_num = re.sub('\D', '', amount)
    return int(amount_num)

In [11]:
## Function to launch scrapper on a specific webpage with number of pages to scrap
def pages_scrap(main_page, iter_page, pages):
    max_pages = (page_max(main_page)/18)
    list_of_pages = page_list(iter_page, pages) # Create list of pages to scrape
    df_list = [] #Create list of dataframes to be concatenated by the end of the loop
    
    # Loop through all pages to create the different dataframes
    for page in list_of_pages:
        html_page = requests.get(page)
        html_page.encoding = 'utf-8'
        page = html_page.text
        df_ads = feature_extract(page)
        df_list.append(df_ads)
    
    # Concatenate the different dataframes
    df_results = pd.concat(df_list)
    df_results = df_results.drop_duplicates()
#     df_results = df_results.reset_index(drop=True)
    
    print('Remaining number of page: ', int(max_pages - pages) )
    
    return df_results

1.2 Subway related functions

In [12]:
## Function to extract subway info list from a house webpage on dexter
def get_info_subway(link):
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    subway = soup.find('ul', class_ = 'list-information').text
    
    return subway

In [13]:
## Function to get list of values for subway distances with string

def sub_values(string):
    split = string.split('\n')
    list_1 = list(filter(None, split))
    
    list_2 = []
    for i in list_1:
        x = i.split('-')
        list_2.append(x)

    list_3 = [item.strip() for sublist in list_2 for item in sublist]
    list_4 = list_3[0:3]
    
    return list_3

In [14]:
## Function to get the closest stop on the tube if any
def closest_line(list_of_lines):
    j = 0
    nearby_data = []
    for i in range(len(list_of_lines)):
        if list_of_lines[i] == 'London Underground' or list_of_lines[i] in Underground_lines and (j != 1 and i!=0):
#             print(list_of_lines[i-2])
            if (' ' in list_of_lines[i-2]) == False :
                nearby_data.append(list_of_lines[i-3])
                nearby_data.append(list_of_lines[i-2])
                nearby_data.append(list_of_lines[i-1])
                nearby_data.append(list_of_lines[i])
                j = 1
                
                nearby_data[0] = (' '.join(nearby_data[0:2]))
                del nearby_data[1]
            
            else:
                nearby_data.append(list_of_lines[i-2])
                nearby_data.append(list_of_lines[i-1])
                nearby_data.append(list_of_lines[i])
                j = 1

    return nearby_data

In [15]:
## Function to populate datafrmae with closest tube stop name, distance, and related tube line

def subway_per_house(df):
    #Create new empty (NaN) columns in the existing dataframe
    df = df.reindex(columns = df.columns.tolist() + ['tube_stop','tube_dist','tube_line'])
    
    #Loop through all lines of dataframe
    for i in range(len(df)):
        x = df['link'].iloc[i] #Get link of house page to scrape
        subs = get_info_subway(x) #Extract tube line info
        subs_2 = sub_values(subs) #Get list of subway station and distance
        subs_3 = closest_line(subs_2) #Extract closest tube station only
        
        # Populate dataframe if a tubeway station has been found or not
        if len(subs_3)!= 0:
            df['tube_stop'].iloc[i] = subs_3[0]
            df['tube_dist'].iloc[i] = subs_3[1]
            df['tube_line'].iloc[i] = subs_3[2]
        else:
            df['tube_stop'].iloc[i] = np.NaN
            df['tube_dist'].iloc[i] = np.NaN
            df['tube_line'].iloc[i] = np.NaN
    
    df = df.astype(str)
            
    return df

Functions to clean subway output

In [16]:
def get_tube_dist(string):
    string_m = string.split(' ')
    num_val = string_m[-1]
    
    return num_val

In [17]:
def strip_tube(string):
    string_m = string.split(' ')
    string_m = string_m[:-1]
    string_m = ' '.join(string_m)
    
    return string_m

In [18]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [19]:
## Function to clean subway stops when too many words in the string

def clean_tube_stop_string(string):
    forbiddden_words = ['London Overground', 'Railway', 'Network Rail', 'Tramlink']
    count_forbidden = 0

    for j in forbiddden_words:
        if count_forbidden == 0:
            if j in string:
                string_update = string.split()[-1]
                count_forbidden = 1
            else:
                string_update = string
            
    return(string_update)

In [20]:
## Function to input tube distance into the right column when value is in 'tube_stop'

def clean_tube_dist(df):
    df['tube_dist'] = df['tube_dist'].astype('str')
    
    errors  = df[df.loc[:, 'tube_dist'].map(hasNumbers) == False].copy()
    errors_2 = errors.loc[errors['tube_stop'] != 'nan'].copy()
    errors_2.loc[:, 'tube_dist'] = errors_2.loc[:, 'tube_stop'].map(get_tube_dist)
    errors_2.loc[:, 'tube_stop'] = errors_2.loc[:, 'tube_stop'].map(strip_tube)
    errors_2
    
    #Create copy of original df for modification
    df_copy = df.copy()

    # replace values in final df
    for i in errors_2.index:
        df_copy.loc[i] = errors_2.loc[i]
        
    return df_copy

In [21]:
## Functions to deals with Victoria tube stops (VIctoria being both a tube stop and a line)

def victoria_clean_stop(string):
    str_vic = 'Victoria'
    str_check = string.split()
    if str_check[0] == 'Victoria':
        str_return = str_check[1]
    else:
        str_return = str_vic

    return str_return

In [22]:
def clean_tube_victoria(df):
    df['tube_stop'] = df['tube_stop'].astype('str')
    
    errors  = df[df['tube_stop'].str.contains('Victoria')].copy()
    
    errors.loc[:, 'tube_stop'] = errors.loc[:, 'tube_stop'].map(victoria_clean_stop)

    #Create copy of original df for modification
    df_copy = df.copy()

    # replace values in final df
    for i in errors.index:
        df_copy.loc[i] = errors.loc[i]
        
    return df_copy

In [31]:
## Final cleaning function to appy previous cleaning on 'tube_stop' and 'tube_dist' for the whole dataframe

def clean_tube_stop(df):
    df_2 = df.copy()
    df_2 = clean_tube_dist(df_2)
    df_2['tube_stop'] = df_2['tube_stop'].astype('str')
    df_2['tube_stop'] = df_2['tube_stop'].map(clean_tube_stop_string)
    
    df_2 = clean_tube_victoria(df_2)
    
    df_2['price'] = df_2['price'].str.replace('£', '')
    df_2['price'] = df_2['price'].str.replace(',', '').astype(float)
    df_2['tube_dist'] = df_2['tube_dist'].str.replace('m', '').astype(float)
    
    
    return df_2

1.3 Execution on a sample set of adds

In [24]:
%%time
## pages_scrap(base_link, pages_link, #number of pages to scrap)


dexter_list_1 = pages_scrap('https://www.dexters.co.uk/property-sales/properties-for-sale-in-london',
                            'https://www.dexters.co.uk/property-sales/properties-for-sale-in-london/page-', 2)

Remaining number of page:  254
Wall time: 3.88 s


In [25]:
dexter_list_1

,ad_identification,name,price,address,bedrooms,bathrooms,reception,link
1,151907,Lancaster Gate,"£25,000,000","Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...
2,124656,Wilton Crescent,"£19,500,000","Belgravia, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...
3,138546,Whitehall Place,"£9,250,000","Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...
4,143179,Old Queen Street,"£9,250,000","Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...
5,124663,Wilton Place,"£9,000,000","Belgravia, SW1X",5,5,3,https://www.dexters.co.uk/property-for-sale/ho...
6,138545,Whitehall Place,"£9,000,000","Whitehall, SW1A",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...
7,138542,Whitehall Place,"£8,500,000","Whitehall, SW1A",2,2,2,https://www.dexters.co.uk/property-for-sale/fl...
8,148090,Ennismore Gardens,"£6,750,000","South Kensington, SW7",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...
9,155928,Radnor Terrace,"£6,500,000","Kensington, W14",4,4,1,https://www.dexters.co.uk/property-for-sale/fl...
10,157170,Welbeck Street,"£6,500,000","Marylebone, W1G",7,7,4,https://www.dexters.co.uk/property-for-sale/ho...


In [26]:
%%time

## Fetch subway related information from the previous dataframe
output_list = subway_per_house(dexter_list_1)

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Wall time: 17 s


In [27]:
output_list

,ad_identification,name,price,address,bedrooms,bathrooms,reception,link,tube_stop,tube_dist,tube_line
1,151907,Lancaster Gate,"£25,000,000","Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26m,Central
2,124656,Wilton Crescent,"£19,500,000","Belgravia, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Network Rail Knightsbridge,0.22m,Piccadilly
3,138546,Whitehall Place,"£9,250,000","Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
4,143179,Old Queen Street,"£9,250,000","Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16m,Circle
5,124663,Wilton Place,"£9,000,000","Belgravia, SW1X",5,5,3,https://www.dexters.co.uk/property-for-sale/ho...,Network Rail Knightsbridge,0.19m,Piccadilly
6,138545,Whitehall Place,"£9,000,000","Whitehall, SW1A",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
7,138542,Whitehall Place,"£8,500,000","Whitehall, SW1A",2,2,2,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12m,Bakerloo
8,148090,Ennismore Gardens,"£6,750,000","South Kensington, SW7",3,3,2,https://www.dexters.co.uk/property-for-sale/fl...,South Kensington,0.38m,District
9,155928,Radnor Terrace,"£6,500,000","Kensington, W14",4,4,1,https://www.dexters.co.uk/property-for-sale/fl...,Kensington (Olympia),0.22m,District
10,157170,Welbeck Street,"£6,500,000","Marylebone, W1G",7,7,4,https://www.dexters.co.uk/property-for-sale/ho...,Bond Street,0.17m,Central


In [33]:
cleaned = clean_tube_stop(output_list)
cleaned.head(5)

,ad_identification,name,price,address,bedrooms,bathrooms,reception,link,tube_stop,tube_dist,tube_line
1,151907,Lancaster Gate,25000000.0,"Hyde Park, W2",60,0,1,https://www.dexters.co.uk/property-for-sale/pr...,Lancaster Gate,0.26,Central
2,124656,Wilton Crescent,19500000.0,"Belgravia, SW1X",5,5,2,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.22,Piccadilly
3,138546,Whitehall Place,9250000.0,"Whitehall, SW1A",3,3,1,https://www.dexters.co.uk/property-for-sale/fl...,Charing Cross,0.12,Bakerloo
4,143179,Old Queen Street,9250000.0,"Westminster, SW1H",5,4,6,https://www.dexters.co.uk/property-for-sale/ho...,St.James's Park,0.16,Circle
5,124663,Wilton Place,9000000.0,"Belgravia, SW1X",5,5,3,https://www.dexters.co.uk/property-for-sale/ho...,Knightsbridge,0.19,Piccadilly


# 

# Output to file and reading from file

In [ ]:
cleaned.to_json(r'C:\Users\user\Documents_C\Python\Jupyter\05. Data engineering\Dexter_output_clean.json', orient = 'columns')
cleaned.to_csv(r'C:\Users\user\Documents_C\Python\Jupyter\05. Data engineering\Dexter_output_clean.csv')

In [ ]:
df_json = pd.read_json(r'C:\Users\user\Documents_C\Python\Jupyter\05. Data engineering\Group_Project\Dexter_output_v01.json',
            orient = 'columns')